# Hyper-parameter Tuning Workflow
Another technique very much worth noting is hyper-parameter tuning, which follows a similar structure to the pre-processor tuning but deals with selecting the best model training parameters for the given data.

Unfortunately there is no rigorous way to predict what parameters will be ideal for a given model or dataset, and thus training (and intuition built up over years of practice) is necessary. Still, we can have the computer run the tuning, even if we need to determine the value's ranges and what precisely to tune.

In [ ]:
# Implement learning rate scheduler
# Implement batch normalization
# Implement mixed precision tuning
# Implement model pruning